In [1]:
import pandas as pd
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

In [10]:
training_data = pd.read_csv('train_file.csv')

In [11]:
training_data

,IDLink,Title,Headline,Source,Topic,PublishDate,Facebook,GooglePlus,LinkedIn,SentimentTitle,SentimentHeadline
0,Tr3CMgRv1N,Obama Lays Wreath at Arlington National Cemetery,Obama Lays Wreath at Arlington National Cemete...,USA TODAY,obama,2002-04-02 00:00:00,-1.0,-1.0,-1.0,0.000000,-0.053300
1,Wc81vGp8qZ,A Look at the Health of the Chinese Economy,"Tim Haywood, investment director business-unit...",Bloomberg,economy,2008-09-20 00:00:00,-1.0,-1.0,-1.0,0.208333,-0.156386
2,zNGH03CrZH,Nouriel Roubini: Global Economy Not Back to 2008,"Nouriel Roubini, NYU professor and chairman at...",Bloomberg,economy,2012-01-28 00:00:00,-1.0,-1.0,-1.0,-0.425210,0.139754
3,3sM1H0W8ts,Finland GDP Expands In Q4,Finland's economy expanded marginally in the t...,RTT News,economy,2015-03-01 00:06:00,-1.0,-1.0,-1.0,0.000000,0.026064
4,wUbnxgvqaZ,"Tourism, govt spending buoys Thai economy in J...",Tourism and public spending continued to boost...,The Nation - Thailand&#39;s English news,economy,2015-03-01 00:11:00,-1.0,-1.0,-1.0,0.000000,0.141084
...,...,...,...,...,...,...,...,...,...,...,...
16081,siLByV5vex,"Excalibur Integrated Systems, Inc Announces It...",Chattanooga based Excalibur Integrated Systems...,The Chattanoogan,microsoft,2015-12-18 17:05:23,0.0,0.0,0.0,0.083333,0.000000
16082,LcSwvtetAp,The Microsoft Band 2 is down to just $199,Microsoft and their partner retailers have put...,Phandroid.com,microsoft,2015-12-18 17:05:24,8.0,12.0,0.0,0.007579,-0.113832
16083,Zr0DNoE1Jr,"Microsoft (MSFT) Acquires Data Analytics Firm,...",Today I'm pleased to announce that Microsoft h...,StreetInsider.com (blog),microsoft,2015-12-18 17:06:22,0.0,1.0,0.0,-0.094491,0.027277
16084,EOtvcVei8O,Senate sends $1.8 trillion deal to Obama,The bill will make permanent three core elemen...,The Hill (blog),obama,2015-12-18 17:06:39,156.0,14.0,21.0,0.000000,0.129164


In [12]:
training_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16086 entries, 0 to 16085
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   IDLink             16086 non-null  object 
 1   Title              16086 non-null  object 
 2   Headline           16086 non-null  object 
 3   Source             16057 non-null  object 
 4   Topic              16086 non-null  object 
 5   PublishDate        16086 non-null  object 
 6   Facebook           16085 non-null  float64
 7   GooglePlus         16085 non-null  float64
 8   LinkedIn           16085 non-null  float64
 9   SentimentTitle     16085 non-null  float64
 10  SentimentHeadline  16085 non-null  float64
dtypes: float64(5), object(6)
memory usage: 1.4+ MB


In [13]:
training_data.isna().sum()

IDLink                0
Title                 0
Headline              0
Source               29
Topic                 0
PublishDate           0
Facebook              1
GooglePlus            1
LinkedIn              1
SentimentTitle        1
SentimentHeadline     1
dtype: int64

In [15]:
training_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16086 entries, 0 to 16085
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   IDLink             16086 non-null  object 
 1   Title              16086 non-null  object 
 2   Headline           16086 non-null  object 
 3   Source             16057 non-null  object 
 4   Topic              16086 non-null  object 
 5   PublishDate        16086 non-null  object 
 6   Facebook           16085 non-null  float64
 7   GooglePlus         16085 non-null  float64
 8   LinkedIn           16085 non-null  float64
 9   SentimentTitle     16085 non-null  float64
 10  SentimentHeadline  16085 non-null  float64
dtypes: float64(5), object(6)
memory usage: 1.4+ MB


In [16]:
sample = pd.read_csv('sample.csv')
sample

,IDLink,SentimentTitle,SentimentHeadline
0,CZLVHanbC2,0.000000,-0.014113
1,VFkR6aHMVJ,0.080799,-0.038613
2,P7iEhVkGlM,-0.045645,-0.120476
3,ij5NwOmhCO,0.291667,0.094653
4,AZhpvyzDfC,0.291663,0.094658
5,K68BWTk8nY,0.044194,0.058465
6,m95WZhuovK,0.000000,0.326718
7,4qwM4lsvYm,0.043212,-0.088388
8,u8iGT7DY2a,-0.079057,0.083853
9,3clx8GUpzY,0.000000,0.063732


In [17]:
training_data = training_data[training_data['Facebook'].notna()]
training_data


,IDLink,Title,Headline,Source,Topic,PublishDate,Facebook,GooglePlus,LinkedIn,SentimentTitle,SentimentHeadline
0,Tr3CMgRv1N,Obama Lays Wreath at Arlington National Cemetery,Obama Lays Wreath at Arlington National Cemete...,USA TODAY,obama,2002-04-02 00:00:00,-1.0,-1.0,-1.0,0.000000,-0.053300
1,Wc81vGp8qZ,A Look at the Health of the Chinese Economy,"Tim Haywood, investment director business-unit...",Bloomberg,economy,2008-09-20 00:00:00,-1.0,-1.0,-1.0,0.208333,-0.156386
2,zNGH03CrZH,Nouriel Roubini: Global Economy Not Back to 2008,"Nouriel Roubini, NYU professor and chairman at...",Bloomberg,economy,2012-01-28 00:00:00,-1.0,-1.0,-1.0,-0.425210,0.139754
3,3sM1H0W8ts,Finland GDP Expands In Q4,Finland's economy expanded marginally in the t...,RTT News,economy,2015-03-01 00:06:00,-1.0,-1.0,-1.0,0.000000,0.026064
4,wUbnxgvqaZ,"Tourism, govt spending buoys Thai economy in J...",Tourism and public spending continued to boost...,The Nation - Thailand&#39;s English news,economy,2015-03-01 00:11:00,-1.0,-1.0,-1.0,0.000000,0.141084
...,...,...,...,...,...,...,...,...,...,...,...
16080,RkH6Odfma6,Sources: Obama's Gun Control Will Come After H...,Sources say that President Obama will drop his...,PJ Media,obama,2015-12-18 17:04:41,199.0,4.0,0.0,0.074665,0.055521
16081,siLByV5vex,"Excalibur Integrated Systems, Inc Announces It...",Chattanooga based Excalibur Integrated Systems...,The Chattanoogan,microsoft,2015-12-18 17:05:23,0.0,0.0,0.0,0.083333,0.000000
16082,LcSwvtetAp,The Microsoft Band 2 is down to just $199,Microsoft and their partner retailers have put...,Phandroid.com,microsoft,2015-12-18 17:05:24,8.0,12.0,0.0,0.007579,-0.113832
16083,Zr0DNoE1Jr,"Microsoft (MSFT) Acquires Data Analytics Firm,...",Today I'm pleased to announce that Microsoft h...,StreetInsider.com (blog),microsoft,2015-12-18 17:06:22,0.0,1.0,0.0,-0.094491,0.027277


In [18]:
training_data.isna().sum()

IDLink                0
Title                 0
Headline              0
Source               29
Topic                 0
PublishDate           0
Facebook              0
GooglePlus            0
LinkedIn              0
SentimentTitle        0
SentimentHeadline     0
dtype: int64

In [21]:
training_data = training_data.dropna()
training_data

,IDLink,Title,Headline,Source,Topic,PublishDate,Facebook,GooglePlus,LinkedIn,SentimentTitle,SentimentHeadline
0,Tr3CMgRv1N,Obama Lays Wreath at Arlington National Cemetery,Obama Lays Wreath at Arlington National Cemete...,USA TODAY,obama,2002-04-02 00:00:00,-1.0,-1.0,-1.0,0.000000,-0.053300
1,Wc81vGp8qZ,A Look at the Health of the Chinese Economy,"Tim Haywood, investment director business-unit...",Bloomberg,economy,2008-09-20 00:00:00,-1.0,-1.0,-1.0,0.208333,-0.156386
2,zNGH03CrZH,Nouriel Roubini: Global Economy Not Back to 2008,"Nouriel Roubini, NYU professor and chairman at...",Bloomberg,economy,2012-01-28 00:00:00,-1.0,-1.0,-1.0,-0.425210,0.139754
3,3sM1H0W8ts,Finland GDP Expands In Q4,Finland's economy expanded marginally in the t...,RTT News,economy,2015-03-01 00:06:00,-1.0,-1.0,-1.0,0.000000,0.026064
4,wUbnxgvqaZ,"Tourism, govt spending buoys Thai economy in J...",Tourism and public spending continued to boost...,The Nation - Thailand&#39;s English news,economy,2015-03-01 00:11:00,-1.0,-1.0,-1.0,0.000000,0.141084
...,...,...,...,...,...,...,...,...,...,...,...
16080,RkH6Odfma6,Sources: Obama's Gun Control Will Come After H...,Sources say that President Obama will drop his...,PJ Media,obama,2015-12-18 17:04:41,199.0,4.0,0.0,0.074665,0.055521
16081,siLByV5vex,"Excalibur Integrated Systems, Inc Announces It...",Chattanooga based Excalibur Integrated Systems...,The Chattanoogan,microsoft,2015-12-18 17:05:23,0.0,0.0,0.0,0.083333,0.000000
16082,LcSwvtetAp,The Microsoft Band 2 is down to just $199,Microsoft and their partner retailers have put...,Phandroid.com,microsoft,2015-12-18 17:05:24,8.0,12.0,0.0,0.007579,-0.113832
16083,Zr0DNoE1Jr,"Microsoft (MSFT) Acquires Data Analytics Firm,...",Today I'm pleased to announce that Microsoft h...,StreetInsider.com (blog),microsoft,2015-12-18 17:06:22,0.0,1.0,0.0,-0.094491,0.027277


In [22]:
training_data.isna().sum()

IDLink               0
Title                0
Headline             0
Source               0
Topic                0
PublishDate          0
Facebook             0
GooglePlus           0
LinkedIn             0
SentimentTitle       0
SentimentHeadline    0
dtype: int64

In [23]:
test_data = pd.read_csv('test_file.csv')
test_data

,IDLink,Title,Headline,Source,Topic,PublishDate,Facebook,GooglePlus,LinkedIn
0,tFrqIR6Chj,Sliding Economy: FG fights back with N3trn TSA...,With the 2016 budget now passed by the Nationa...,BusinessDay,economy,2016-03-29 01:41:12,0,0,1
1,DVAaGErjlF,Microsoft shows how HoloLens can bring distant...,A recent Microsoft Research video shows how th...,Daily Mail,microsoft,2016-03-29 01:41:27,121,2,13
2,OT9UIZm5M2,"Microsoft’s Twitter Robot Praises Hitler, Trum...",* Microsoft teamed with Bing to create TayTwee...,EURweb,microsoft,2016-03-29 01:47:00,12,1,0
3,lflGp3q2Fj,Flood of Central Bank Moves Can't Get World Ec...,Central bankers have managed to steer the worl...,Bloomberg via Yahoo! Finance,economy,2016-03-29 02:00:00,0,0,3
4,zDYG0SoovZ,USD/JPY: bears lining up on mixed U.S. economy...,"However, this streak of seven-day gains might ...",FXStreet,economy,2016-03-29 02:01:07,3,0,0
...,...,...,...,...,...,...,...,...,...
37283,5bYaKaEyN3,Stocks rise as investors key in on US economy ...,The June employment report is viewed as a cruc...,MarketWatch,economy,2016-07-07 15:31:05,-1,3,5
37284,jmnC32Uh5u,Russian PM proposes to use conservative and to...,"In addition, establish stimulating economic po...",TASS,economy,2016-07-07 15:31:10,-1,0,1
37285,yPhgBwobV0,Palestinian Government Uses Foreign Aid To Pay...,The Palestinian government spends nearly $140 ...,Daily Caller,palestine,2016-07-07 15:38:26,5,1,0
37286,adFcODOaiQ,Palestine Youth Orchestra prepares for first U...,Palestine Youth Orchestra prepares for first U...,Ahram Online,palestine,2016-07-07 15:59:22,0,0,0
